In [1]:
import unicodedata
import csv
import random
from openai import OpenAI
from sklearn.metrics import mean_absolute_error
import numpy as np

sample_num = 100
example_num = 4
content_truncate = 150

In [2]:
csv_name = 'Multi-qm9.csv'
data = []
with open(csv_name, 'r') as file:
    reader = csv.reader(file)
    headers = next(reader)  
    for row in reader:
        data.append(tuple(row))
data_set = set(data)

data_sample = random.sample(data, sample_num)
data_sample_set = set(data_sample)
example_set = data_set - data_sample_set
sample_example = random.sample(list(example_set), example_num * sample_num)

sample_example = np.array(sample_example).reshape(sample_num, example_num, len(headers))

print(data_sample)

[('gdb_49', 'CC(=NO)C', '8.63483', '4.11977', '2.88825', '0.6875', '45.37', '-0.2392', '0.0192', '0.2584', '452.5112', '0.100501', '-248.375248', '-248.368823', '-248.367879', '-248.405354', '21.616', '-1057.089739', '-1063.722509', '-1070.242955', '-982.9834359', '67180', 'This molecule is a ketoxime.'), ('gdb_164', 'C1CC2OC12', '9.60877', '6.57366', '5.00982', '1.8518', '41.8', '-0.2517', '0.0951', '0.3468', '317.7261', '0.091787', '-231.102112', '-231.097923', '-231.096979', '-231.128841', '15.427', '-1034.641232', '-1041.789187', '-1047.716637', '-965.4094188', '12621600', 'CID is 12621600,compound_name is 5-Oxabicyclo[2.1.0]pentane,cid_paras is 12621600,Molecular_Weight is 70.09,XLogP3 is 0.5,Hydrogen_Bond_Donor_Count is 0,Hydrogen_Bond_Acceptor_Count is 1,Rotatable_Bond_Count is 0,Exact_Mass is 70.041864811,Monoisotopic_Mass is 70.041864811,Topological_Polar_Surface_Area is 12.5,"Unit":"??",Heavy_Atom_Count is 5,Formal_Charge is 0,Complexity is 52.7,Isotope_Atom_Count is 0,Define

In [3]:
mol_id_index = headers.index('mol_id')
smiles_index = headers.index('smiles')
A_index = headers.index('A')
B_index = headers.index('B')
C_index = headers.index('C')
description_index = headers.index('Description')
mu_index = headers.index('mu')
alpha_index = headers.index('alpha')

contents = []
mus_true = []
alphas_true = []
for data_i in range(len(data_sample)):
    data = data_sample[data_i]
    content = "Molecule ID: " + data[mol_id_index] + "\n"
    content += "SMILES string: " + data[smiles_index] + "\n"
    content += "Coordinates: " + data[A_index] + ", " + data[B_index] + ", " + data[C_index] + "\n"
    content += "Description: " + data[description_index][:content_truncate] + ".\n\n"
    content += "You are an expert chemist with a comprehensive understanding of molecular structures and chemistry. Please predict the Dipole Moment (mu) and Isotropic Polarizability (alpha) of the given molecule.\n"
    content += "Here are some examples:\n"
    example = sample_example[data_i]

    for example_i in range(example_num):
        example_data = example[example_i]
        content += "Example " + str(example_i + 1) + ":\n"
        content += "Molecule ID: " + example_data[mol_id_index] + "\n"
        content += "SMILES string: " + example_data[smiles_index] + "\n"
        content += "Coordinates: " + example_data[A_index] + ", " + example_data[B_index] + ", " + example_data[C_index] + "\n"
        content += "Description: " + example_data[description_index][:content_truncate] + ".\n"
        content += "Dipole Moment (mu): " + example_data[mu_index] + "\n"
        content += "Isotropic Polarizability (alpha): " + example_data[alpha_index] + "\n\n"
    
    content += "Provide your answer of the molecule in the format: 'mu: 1.23, alpha: 4.56' and avoid any additional information."
    contents.append(content)
    mus_true.append(float(data[mu_index]))
    alphas_true.append(float(data[alpha_index]))

print(contents[0])

Molecule ID: gdb_49
SMILES string: CC(=NO)C
Coordinates: 8.63483, 4.11977, 2.88825
Description: This molecule is a ketoxime..

You are an expert chemist with a comprehensive understanding of molecular structures and chemistry. Please predict the Dipole Moment (mu) and Isotropic Polarizability (alpha) of the given molecule.
Here are some examples:
Example 1:
Molecule ID: gdb_37
SMILES string: COC=O
Coordinates: 48.62351, 4.67298, 4.38187
Description: This molecule appears as a clear colorless liquid with an agreeable odor. Flash point -27 °F. Less dense than water Vapors heavier than air., This mol.
Dipole Moment (mu): 3.92
Isotropic Polarizability (alpha): 29.47

Example 2:
Molecule ID: gdb_451
SMILES string: C1COCC1=O
Coordinates: 7.15925, 3.46557, 2.4869
Description: This molecule is an oxalane substituted by an oxo group at position 3. It is functionally related to an oxolane..
Dipole Moment (mu): 1.8765
Isotropic Polarizability (alpha): 46.04

Example 3:
Molecule ID: gdb_233
SMILES

In [4]:
client = OpenAI(
    api_key="your api key"
)

answer_contents = []
for content in contents:
    completion = client.chat.completions.create(
        messages=[
        {"role": "system", "content": "You are an experienced chemist."},
        {"role": "user", "content": content}
        ],
        model="gpt-3.5-turbo-1106",
    )

    message = completion.choices[0].message
    message_content = unicodedata.normalize('NFKC', message.content)
    print('message_content:', message_content)

    answer_contents.append(message_content)



message_content: mu: 0.0, alpha: 0.0
message_content: mu: 0.0, alpha: 0.0
message_content: mu: 3.421, alpha: 9.87
message_content: mu: 0
alpha: 0
message_content: mu: 1.84, alpha: 24.45
message_content: mu: 0, alpha: 0
message_content: mu: 1.72, alpha: 24.35
message_content: mu: 0.0, alpha: 28.83
message_content: mu: 1.98, alpha: 9.34
message_content: mu: 0.36, alpha: 2.07
message_content: mu: 0.2278, alpha: 2.84
message_content: mu: 1.86, alpha: 5.67
message_content: mu: 1.0556, alpha: 2.2946
message_content: Since the molecule C1COC1 is symmetrical, it has a dipole moment of zero (mu: 0.00) and an isotropic polarizability of zero (alpha: 0.00).
message_content: mu: 0, alpha: 24.59
message_content: mu: 2.31, alpha: 40.72
message_content: mu: 0.00, alpha: 0.00
message_content: mu: 2.03, alpha: 24.82
message_content: mu: 0.0, alpha: 0.0
message_content: mu: 0.0, alpha: 16.18
message_content: mu: 2.15, alpha: 46.39
message_content: mu: 0.0, alpha: 0.0
message_content: mu: 2.4648, alpha: 

In [5]:
import re

print(len(answer_contents))
def judge_content(content):
    pattern = r'^mu: \d+(\.\d+)?, alpha: \d+(\.\d+)?$'
    return bool(re.match(pattern, content))

def extract_values(s):
    parts = s.split(',')
    mu = float(parts[0].split(':')[1].strip())
    alpha = float(parts[1].split(':')[1].strip())
    return mu, alpha

mus_pred = []
alphas_pred = []
for answer_i in range(len(answer_contents)):
    answer = answer_contents[answer_i]
    if not judge_content(answer):
        print('Invalid answer:', answer_i)
        mus_pred.append(-1)
        alphas_pred.append(-1)
        continue

    mu, alpha = extract_values(answer)
    mus_pred.append(mu)
    alphas_pred.append(alpha)

indices = [i for i, x in enumerate(mus_pred) if x != -1]
# print(indices)
mus_true = [mus_true[i] for i in indices]
mus_pred = [mus_pred[i] for i in indices]
alphas_true = [alphas_true[i] for i in indices]
alphas_pred = [alphas_pred[i] for i in indices]
# print(len(mus_true), len(mus_pred), len(alphas_true), len(alphas_pred))
# print(mus_pred)
    
mu_mae = mean_absolute_error(mus_true, mus_pred)
alpha_mae = mean_absolute_error(alphas_true, alphas_pred)
print('mu_mae:', mu_mae)
print('alpha_mae:', alpha_mae)


100
Invalid answer: 3
Invalid answer: 13
Invalid answer: 28
Invalid answer: 38
Invalid answer: 51
[0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 36, 37, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
95 95 95 95
[0.0, 0.0, 3.421, 1.84, 0.0, 1.72, 0.0, 1.98, 0.36, 0.2278, 1.86, 1.0556, 0.0, 2.31, 0.0, 2.03, 0.0, 0.0, 2.15, 0.0, 2.4648, 0.0, 1.22, 0.0, 0.0, 0.0, 2.01, 1.22, 0.0, 2.162, 1.02, 0.0, 0.0, 1.85, 0.0, 0.0, 1.87, 1.42, 2.34, 0.0, 0.0, 1.63, 0.2731, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5945, 1.88, 0.0, 0.0, 0.0, 1.357, 2.01, 1.68, 0.0, 0.9421, 0.0, 1.35, 2.65, 2.35, 0.0, 3.42, 1.45, 0.0, 0.0, 2.7256, 0.0, 0.0, 2.1744, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5583, 1.95, 2.53, 2.9, 0.0, 2.15, 1.94048, 1.062, 0.0, 2.8734, 1.37, 2.29, 2.372